### Загрузка данных

In [1]:
#Импортируем необходимые библиотеки

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pymongo import MongoClient
import re

In [2]:
#Устанавливаем соединение с клиентом Mongo

client = MongoClient(host='localhost', port=27017)

In [3]:
#Выводим список баз данных, размещенных на сервере

dbs = client.list_database_names()
dbs

['2000_kinopoisk_top_300',
 '2001_kinopoisk_top_300',
 '2002_kinopoisk_top_300',
 '2003_kinopoisk_top_300',
 '2004_kinopoisk_top_300',
 '2005_kinopoisk_top_300',
 '2006_kinopoisk_top_300',
 '2007_kinopoisk_top_300',
 '2008_kinopoisk_top_300',
 '2009_kinopoisk_top_300',
 '2010_kinopoisk_top_300',
 '2011_kinopoisk_top_300',
 '2012_kinopoisk_top_300',
 '2013_kinopoisk_top_300',
 '2014_kinopoisk_top_300',
 '2015_kinopoisk_top_300',
 '2016_kinopoisk_top_300',
 '2017_kinopoisk_top_300',
 '2018_kinopoisk_top_300',
 '2019_kinopoisk_top_300',
 '2020_kinopoisk_top_300',
 'admin',
 'config',
 'local']

In [4]:
# Сшиваем общий датафрейм из подгруженных баз данных по годам

concat_fl = 0
for db in dbs[:-2]:
    if concat_fl == 0:
        main_df = pd.DataFrame(list(client[db].films.find({})))
        concat_fl = 1
    else:
        main_df = pd.concat([main_df, pd.DataFrame(list(client[db].films.find({})))])
main_df.head()

,_id,film_name,film_time,country,main_roles,genre,director,rating,url,film_year
0,1,Российская империя (сериал),60 мин,"Россия,","\nЛеонид Парфенов, Елизавета Листова, ...\n","документальный, история)\n",Джаник Файзиев,8.8,https://www.kinopoisk.ru//film/453188/sr/1/,2000
1,2,Metallica: S&M (ТВ),145 мин,"США,","\nMetallica, Джеймс Хетфилд, ...\n","документальный, музыка, концерт)\n",Уэйн Ишам,8.6,https://www.kinopoisk.ru//film/337889/sr/1/,2000
2,3,Гладиатор,155 мин,США...,"\nРассел Кроу, Хоакин Феникс, ...\n","история, боевик, драма...)\n",Ридли Скотт,8.6,https://www.kinopoisk.ru//film/474/sr/1/,2000
3,4,Большой куш,104 мин,Великобритания...,"\nДжейсон Стэйтем, Стивен Грэм, ...\n","криминал, комедия, боевик)\n",Гай Ричи,8.5,https://www.kinopoisk.ru//film/526/sr/1/,2000
4,5,Близкие друзья (сериал),45 мин,Канада...,"\nГейл Харольд, Хэл Спаркс, ...\n","драма, мелодрама)\n",Майкл ДеКарло,8.4,https://www.kinopoisk.ru//film/262973/sr/1/,2000


### Обработка данных

In [5]:
# Выводим типы данных в столбцах

main_df.dtypes

_id           object
film_name     object
film_time     object
country       object
main_roles    object
genre         object
director      object
rating        object
url           object
film_year     object
dtype: object

In [6]:
# Убираем колонку "_id", т.к. она нам не требуется, а также обновляем индексирование датафрейма

main_df = main_df.drop(columns={'_id'}).reset_index(drop=True)
main_df.head()

,film_name,film_time,country,main_roles,genre,director,rating,url,film_year
0,Российская империя (сериал),60 мин,"Россия,","\nЛеонид Парфенов, Елизавета Листова, ...\n","документальный, история)\n",Джаник Файзиев,8.8,https://www.kinopoisk.ru//film/453188/sr/1/,2000
1,Metallica: S&M (ТВ),145 мин,"США,","\nMetallica, Джеймс Хетфилд, ...\n","документальный, музыка, концерт)\n",Уэйн Ишам,8.6,https://www.kinopoisk.ru//film/337889/sr/1/,2000
2,Гладиатор,155 мин,США...,"\nРассел Кроу, Хоакин Феникс, ...\n","история, боевик, драма...)\n",Ридли Скотт,8.6,https://www.kinopoisk.ru//film/474/sr/1/,2000
3,Большой куш,104 мин,Великобритания...,"\nДжейсон Стэйтем, Стивен Грэм, ...\n","криминал, комедия, боевик)\n",Гай Ричи,8.5,https://www.kinopoisk.ru//film/526/sr/1/,2000
4,Близкие друзья (сериал),45 мин,Канада...,"\nГейл Харольд, Хэл Спаркс, ...\n","драма, мелодрама)\n",Майкл ДеКарло,8.4,https://www.kinopoisk.ru//film/262973/sr/1/,2000


In [7]:
# Проверим датафрейм на наличие пропущенных значений

main_df.isnull().sum()

film_name      0
film_time      0
country        0
main_roles     0
genre         10
director      95
rating        70
url            0
film_year      2
dtype: int64

In [8]:
# Уберем записи, в которых имеются пропуски

main_df = main_df.dropna()

In [9]:
# Выполним повторную проверку

main_df.isnull().sum()

film_name     0
film_time     0
country       0
main_roles    0
genre         0
director      0
rating        0
url           0
film_year     0
dtype: int64

In [10]:
main_df.shape

(6147, 9)

In [11]:
# Приведем данные о времени фильма из строкового в целочисленный формат

main_df.film_time = main_df.film_time.apply(lambda c: c.split(' ')[0])

In [12]:
main_df.film_time.unique()

array(['60', '145', '155', '104', '45', '23', '8', '143', '30', '25', '3',
       '216', '366', '24', '127', '52', '100', '123', '129', '59', '53',
       '43', '157', '102', '185', '158', '113', '376', '140', '162', '10',
       '165', '110', '153', '107', '360', '172', '101', '173', '98', '78',
       '22', '131', '75', '119', '121', '108', '151', '175', '85', '126',
       '6', '29', '89', '84', '103', '99', '120', '125', '80', '105',
       '96', '88', '82', '124', '114', '81', '135', '195', '115', '118',
       '97', '106', '116', '4', '15', '109', '87', '130', '50', '624',
       '94', '76', '93', '132', '11', '26', '112', '111', '28', '147',
       '265', '177', '232', '90', '122', '92', '9', '180', '128', '133',
       '154', '86', '91', '117', '5', '73', '77', '95', '74', '70', '134',
       '389', '178', '210', '150', '152', '160', '42', '176', '7', '146',
       '183', '55', '170', '83', '200', '144', '142', '44', '14', '194',
       '300', '21', '51', '141', '65', '520', '6

In [13]:
# Оказалось, что в некоторых записях отсутствуют данные о времени (по причине отсутствия в источнике)
# Удалим, эти записи

main_df.film_time = pd.to_numeric(main_df.film_time, errors='coerce')
main_df = main_df.dropna()
main_df.film_time = main_df.film_time.astype('int')

In [14]:
#Т.к. мы убрали 'мин' из значений, переименуем колонку "film_time" в "film_time_min"

main_df = main_df.rename(columns={'film_time': 'film_time_min'})

In [15]:
# Приведем в порядок колонку со странами

main_df.country = main_df.country.apply(lambda c: c.replace('... ', '').replace(', ', ''))

In [16]:
# Проверим корректность полученных стран

main_df.country.unique()

array(['Россия', 'США', 'Великобритания', 'Канада', 'Япония', 'Германия',
       'Индия', 'Франция', 'Корея Южная', 'Дания', 'Мексика', 'Италия',
       'Гонконг', 'Нидерланды', 'Венгрия', 'Чехия', 'Австралия', 'Швеция',
       'Тайвань', 'Новая Зеландия', 'Аргентина', 'Украина', 'Польша',
       'Исландия', 'Бразилия', 'Китай', 'Бельгия', 'Испания', 'Швейцария',
       'Ирландия', 'Иран', 'Венесуэла', 'Финляндия', 'Турция', 'Норвегия',
       'Хорватия', 'Колумбия', 'Таиланд', 'Греция', 'Армения', 'Сербия',
       'Израиль', 'Тунис', 'Румыния', 'Узбекистан', 'Австрия', 'Грузия',
       'Эстония', 'Казахстан', 'ЮАР', 'Беларусь', 'Индонезия', 'Марокко',
       'Саудовская Аравия', 'Люксембург', 'Кипр', 'Чили', 'Кения',
       'Бангладеш', 'Латвия', 'Киргизия', ' ', 'Ливан'], dtype=object)

In [17]:
# Видим некорректное название страны ''. Найдем это значение. 

main_df.query('country == " "')

,film_name,film_time_min,country,main_roles,genre,director,rating,url,film_year
5384,Еда по-советски,52,,\n,документальный)\n,Борис Караджев,7.2,https://www.kinopoisk.ru//film/1233201/sr/1/,2017
6174,Восход Османской империи (мини-сериал),44,,"\nCem Yigit Uzümoglu, Tommaso Basili, ...\n","документальный, драма, военный...)\n",Эмре Сахин,7.5,https://www.kinopoisk.ru//film/1244945/sr/1/,2020


In [18]:
# Укажем страны-производителей данных фильмов

main_df.loc[(main_df.film_name == 'Еда по-советски'), 'country'] = 'Россия'
main_df.loc[(main_df.film_name == 'Восход Османской империи (мини-сериал)'), 'country'] = 'Турция'

In [19]:
# Приведем в порядок колонку с актерами

main_df.main_roles = main_df.main_roles.apply(lambda c: c.replace('\n', '').replace(', ...', ''))
main_df.head()

,film_name,film_time_min,country,main_roles,genre,director,rating,url,film_year
0,Российская империя (сериал),60,Россия,"Леонид Парфенов, Елизавета Листова","документальный, история)\n",Джаник Файзиев,8.8,https://www.kinopoisk.ru//film/453188/sr/1/,2000
1,Metallica: S&M (ТВ),145,США,"Metallica, Джеймс Хетфилд","документальный, музыка, концерт)\n",Уэйн Ишам,8.6,https://www.kinopoisk.ru//film/337889/sr/1/,2000
2,Гладиатор,155,США,"Рассел Кроу, Хоакин Феникс","история, боевик, драма...)\n",Ридли Скотт,8.6,https://www.kinopoisk.ru//film/474/sr/1/,2000
3,Большой куш,104,Великобритания,"Джейсон Стэйтем, Стивен Грэм","криминал, комедия, боевик)\n",Гай Ричи,8.5,https://www.kinopoisk.ru//film/526/sr/1/,2000
4,Близкие друзья (сериал),45,Канада,"Гейл Харольд, Хэл Спаркс","драма, мелодрама)\n",Майкл ДеКарло,8.4,https://www.kinopoisk.ru//film/262973/sr/1/,2000


In [20]:
# После удаления ненужных символов, у нас могли остаться пустые строки в столбце 'main_roles'

main_df.query('main_roles == ""').head()

,film_name,film_time_min,country,main_roles,genre,director,rating,url,film_year
31,Адажио,10,Россия,,"мультфильм, короткометражка)\n",Гарри Бардин,7.9,https://www.kinopoisk.ru//film/255327/sr/1/,2000
36,Отец и дочь,8,Великобритания,,"мультфильм, короткометражка, драма)\n",Майкл Дадок де Уит,7.9,https://www.kinopoisk.ru//film/120540/sr/1/,2000
68,D: Жажда крови,103,Япония,,"аниме, мультфильм, фантастика...)\n",Ёсиаки Кавадзири,7.7,https://www.kinopoisk.ru//film/8236/sr/1/,2000
208,Hua yang de nian hua,3,Гонконг,,"короткометражка, мелодрама, военный)\n",Вонг Кар-Вай,7.0,https://www.kinopoisk.ru//film/22852/sr/1/,2000
363,Кошачий суп,30,Япония,,"аниме, мультфильм, короткометражка...)\n",Тацуо Сато,7.7,https://www.kinopoisk.ru//film/321394/sr/1/,2001


In [21]:
# Избавимся от них

main_df = main_df.query('main_roles != ""')

In [22]:
# Приведем в порядок колонку с жанром

main_df.genre = main_df.genre.apply(lambda c: c.replace(')\n', '').replace('...', '').replace(' ', ''))
main_df.head()

,film_name,film_time_min,country,main_roles,genre,director,rating,url,film_year
0,Российская империя (сериал),60,Россия,"Леонид Парфенов, Елизавета Листова","документальный,история",Джаник Файзиев,8.8,https://www.kinopoisk.ru//film/453188/sr/1/,2000
1,Metallica: S&M (ТВ),145,США,"Metallica, Джеймс Хетфилд","документальный,музыка,концерт",Уэйн Ишам,8.6,https://www.kinopoisk.ru//film/337889/sr/1/,2000
2,Гладиатор,155,США,"Рассел Кроу, Хоакин Феникс","история,боевик,драма",Ридли Скотт,8.6,https://www.kinopoisk.ru//film/474/sr/1/,2000
3,Большой куш,104,Великобритания,"Джейсон Стэйтем, Стивен Грэм","криминал,комедия,боевик",Гай Ричи,8.5,https://www.kinopoisk.ru//film/526/sr/1/,2000
4,Близкие друзья (сериал),45,Канада,"Гейл Харольд, Хэл Спаркс","драма,мелодрама",Майкл ДеКарло,8.4,https://www.kinopoisk.ru//film/262973/sr/1/,2000


In [23]:
# Приведем колонку c рейтингом к типу "float"

main_df.rating = main_df.rating.astype('float')

In [24]:
# Приведем колонку c годом к типу "int"

main_df.film_year = main_df.film_year.astype('int')

In [25]:
# После всех преобразований, еще раз проверим типы данных в нашем Датафрейме

main_df.dtypes

film_name         object
film_time_min      int32
country           object
main_roles        object
genre             object
director          object
rating           float64
url               object
film_year          int32
dtype: object

In [26]:
# Выведем результирующий Датафрейм, с обновленным индексом

main_df = main_df.reset_index(drop=True)
main_df

,film_name,film_time_min,country,main_roles,genre,director,rating,url,film_year
0,Российская империя (сериал),60,Россия,"Леонид Парфенов, Елизавета Листова","документальный,история",Джаник Файзиев,8.8,https://www.kinopoisk.ru//film/453188/sr/1/,2000
1,Metallica: S&M (ТВ),145,США,"Metallica, Джеймс Хетфилд","документальный,музыка,концерт",Уэйн Ишам,8.6,https://www.kinopoisk.ru//film/337889/sr/1/,2000
2,Гладиатор,155,США,"Рассел Кроу, Хоакин Феникс","история,боевик,драма",Ридли Скотт,8.6,https://www.kinopoisk.ru//film/474/sr/1/,2000
3,Большой куш,104,Великобритания,"Джейсон Стэйтем, Стивен Грэм","криминал,комедия,боевик",Гай Ричи,8.5,https://www.kinopoisk.ru//film/526/sr/1/,2000
4,Близкие друзья (сериал),45,Канада,"Гейл Харольд, Хэл Спаркс","драма,мелодрама",Майкл ДеКарло,8.4,https://www.kinopoisk.ru//film/262973/sr/1/,2000
...,...,...,...,...,...,...,...,...,...
5927,Рассказы из Петли (сериал),60,США,"Дэниэл Золгадри, Пол Шнайдер","драма,фантастика,мелодрама",Джоди Фостер,7.1,https://www.kinopoisk.ru//film/1178337/sr/1/,2020
5928,Метод 2 (сериал),50,Россия,"Паулина Андреева, Константин Хабенский","триллер,детектив",Александр Войтинский,7.1,https://www.kinopoisk.ru//film/1238997/sr/1/,2020
5929,Моя реинкарнация в отомэ-игре в качестве главн...,24,Япония,"Маая Утида, Сёта Аои","аниме,мультфильм,мелодрама",Кэисукэ Иноэ,7.1,https://www.kinopoisk.ru//film/1313574/sr/1/,2020
5930,Давай найдем друг друга (сериал),50,Россия,"Роман Маякин, Иван Стебунов",мелодрама,Дмитрий Черкасов,7.0,https://www.kinopoisk.ru//film/1363088/sr/1/,2020


### Сохранение подготовленных данных

In [27]:
main_df.to_csv('kinopoisk_prepared.csv', sep=';', index=False)